In [14]:
import numpy as np
import pandas as pd
import torch

In [15]:
df=pd.read_excel("QA.xlsx")

In [16]:
df.head()

,Question,Response
0,How do I access the engine?,There is a generic user login and home page a...
1,How can I access the engine?,There is a generic user login and home page av...
2,How do I get to the engine?,There is a generic user login and home page a...
3,what can I do to access the Aitek engine ?,There is a generic user login and home page a...
4,How do I open the engine?,There is a generic user login and home page a...


In [17]:
from transformers import AutoTokenizer, AutoModel

In [18]:
# Load the pre-trained FLAN-T5 model and tokenizer
model_path = "FLAN-T5 model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path)

Some weights of the model checkpoint at FLAN-T5 model were not used when initializing LongformerModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
# Generate sentence embeddings for the dataset
corpus = df['Question'].tolist()
corpus_embeddings = []
batch_size = 20
for i in range(0, len(corpus), batch_size):
    batch = corpus[i:i+batch_size]
    inputs = tokenizer.batch_encode_plus(batch, add_special_tokens=True, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        embeddings = model(**inputs)[0][:, 0, :].numpy()
    corpus_embeddings.append(embeddings)
corpus_embeddings = np.concatenate(corpus_embeddings, axis=0)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar_question(user_input, corpus, corpus_embeddings):
    # Encode the user input
    input_ids = torch.tensor([tokenizer.encode(user_input, add_special_tokens=True)])
    with torch.no_grad():
        user_input_embedding = model(input_ids)[0][:, 0, :].numpy()

    # Calculate the cosine similarity between the user input and each sentence in the corpus
    similarities = cosine_similarity(user_input_embedding, corpus_embeddings)

    # Find the index of the most similar sentence
    most_similar_idx = np.argmax(similarities)

    # Return the most similar question
    return corpus[most_similar_idx]

In [23]:
def find_answer(most_similar_question):
    # Find the answer corresponding to the most similar question in the dataset
    answer = df.loc[df['Question'] == most_similar_question, 'Response'].iloc[0]

    # Return the answer
    return answer


In [24]:
def chatbot():
    # Get user input
    user_input = input("USER: ")

    # Find the most similar question in the dataset
    most_similar_question = find_most_similar_question(user_input, corpus, corpus_embeddings)

    # Find the answer corresponding to the most similar question in the dataset
    answer = find_answer(most_similar_question)

    # Return the answer
    print("BOT: " + answer)

In [27]:
chatbot()


USER:   How I change my profile settings


BOT:  A user can see which account is logged in. They have access to basic information and profile actions, for example logging out or tweaking profile settings.
